In [94]:
#  Copyright 2017-2021 Reveal Energy Services, Inc
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
# This file is part of Orchid and related technologies.
#

# Example: Using Pandas to Analyze Volume to First Response (VFR)

This notebook illustrates using the Orchid* Python API and the pandas package to
perform VFR analysis

(*Orchid is a mark of Reveal Energy Services, Inc)

## 0.5 Import packages

The only import needed for the Python API is `orchid` itself.

In [95]:
import orchid

The remaining imports are standard python packages to support the analysis.

In [152]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.signal as signal
import datetime
import dateutil.tz as duz

## 1.0 Load the .ifrac project

The following code simply captures the configured location of the Orchid training data. It is not needed to
use the Orchid Python API itself, but it is used in this example to load well-known data.

In [97]:
orchid_training_data_path = orchid.training_data_path()

In [98]:
permian_project = orchid.load_project(str(orchid_training_data_path.joinpath(
    'Project_frankNstein_Permian_UTM13_FEET.ifrac')))

This function will support taking in an stop time and calculating the treatment aggregates

In [99]:
def compute_stage_treatment_aggregates(treatment_stage, stop_time):
    # These calculations IGNORE all calculation warnings.
    pumped_fluid_volume = orchid.pumped_fluid_volume(treatment_stage,
                                                     treatment_stage.start_time,
                                                     stop_time)[0].magnitude
    total_proppant_mass = orchid.total_proppant_mass(treatment_stage,
                                                     treatment_stage.start_time,
                                                     stop_time)[0].magnitude
    median_treating_pressure = orchid.median_treating_pressure(treatment_stage,
                                                               treatment_stage.start_time,
                                                               stop_time)[0].magnitude
    return pumped_fluid_volume, total_proppant_mass, median_treating_pressure

The first event detection algorithm is to find when the first derivative exceeds a threshold value

In [100]:
def first_derivative_threshold(pressure_curve, threshold, window=51, poly=3):
    """
    The
    Args:
        pressure_curve: pressure data from the monitoring well.
                        Series data, index is time, value is a orchid pressure measurement
                        Only pass in the pressure curve data during stage of interest
        threshold: provided a
        window: passed to window_length arg in savgol filter
        poly: passed to polyorder arg in savgol filter

    Returns:

    """
    times = pressure_curve.index.values
    average_dt = np.mean(np.diff(pressure_curve.index)).total_seconds()/60
    pressure = pressure_curve.values
    first_derivative = signal.savgol_filter(pressure,
                                            window_length=window,
                                            polyorder=poly,
                                            delta=average_dt,
                                            deriv=1)

    ndx = np.argwhere(first_derivative > threshold)

    return times[ndx[0][0]] if len(ndx)>0 else None

In [104]:
def second_derivative_peak(pressure_curve, threshold, window=51, poly=3):
    times = pressure_curve.index.values
    average_dt = np.mean(np.diff(pressure_curve.index)).total_seconds()/60
    pressure = pressure_curve.values
    second_derivative = signal.savgol_filter(pressure,
                                             window_length=window,
                                             polyorder=poly,
                                             delta=average_dt,
                                             deriv=2)

    peaks = signal.find_peaks(second_derivative)
    return times[peaks[0][0]]

In [105]:
p_curves = list(permian_project.monitor_curves())
p_curve = p_curves[0]
p_data = p_curve.time_series()

In [155]:
data = []
for well in permian_project.wells:
    for stage in well.stages:
        stage_start_time = stage.start_time
        stage_stop_time = stage.stop_time

        p_stg = p_data[stage_start_time:stage_stop_time]
        d1 = first_derivative_threshold(p_stg, 0.2)
        d2 = second_derivative_peak(p_stg, 1)
        if d1 is not None:
            d1 = datetime.datetime.utcfromtimestamp(d1.tolist() * 1e-9).replace(tzinfo=duz.UTC)
        if d2 is not None:
            d2 = datetime.datetime.utcfromtimestamp(d2.tolist() * 1e-9).replace(tzinfo=duz.UTC)
        vfr_d1, _, _ = compute_stage_treatment_aggregates(stage, d1) if d1 is not None else (None, None, None)
        vfr_d2, _, _ = compute_stage_treatment_aggregates(stage, d2) if d2 is not None else (None, None, None)
        data.append((well.name, stage.display_stage_number, d1, d2, vfr_d1, vfr_d2))




In [156]:
df = pd.DataFrame(data=data, columns=['Well', 'Stage', 'D1', 'D2', 'VFR_D1', 'VFR_D2'])

In [157]:
df.head()

,Well,Stage,D1,D2,VFR_D1,VFR_D2
0,C1,1,NaT,2018-11-13 20:23:50+00:00,NaN,0.0
1,C1,2,NaT,2018-11-14 21:58:16+00:00,NaN,0.0
2,C1,3,2018-11-15 06:12:26+00:00,2018-11-15 04:36:22+00:00,4690.6245,0.0
3,C1,4,2018-11-15 14:38:54+00:00,2018-11-15 12:56:24+00:00,5201.4615,0.0
4,C1,5,NaT,2018-11-15 20:21:40+00:00,NaN,0.0
